In [2]:
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlapp import SqlApp
import enum
from functools import partial


In [3]:
# PTO IMPORT
from web.datalayer.partner_ref import AvailableCountryCodes
from web.datalayer.organization import Organization
from web.datalayer.business_partners_sql import OrganizationPublisher
from web.datalayer.publisher import Publisher, PublisherField
from web.datalayer.country_code import CountryCode

In [4]:
sqlapp = SqlApp()
sqlapp.connect('postgres://postgres@localhost:5432/partoo', echo=False)
populate, create, Base = map(partial(getattr, sqlapp), ['populate', 'create', 'Base'])

In [5]:
## HELPERS
def _pop(_dict, keys, default=None):
    props = map(partial(lambda p, x: _dict.pop(x, p), default), keys)
    return props if len(keys) > 1 else props[0]

def _get(_dict, keys):
    props = map(_dict.get, keys)
    return props if len(keys) > 1 else props[0]

In [6]:
def updatePublisher(id, **kwargs):
    # fetch record:
    publisher = sqlapp.session.query(Publisher).filter_by(name=id).first()

    # extract relation fields:
    organizations, country_codes, pm_fields = _pop(kwargs, ['organizations', 'country_codes', 'pm_fields'])
    
    if publisher:
        for k,v in kwargs.items():
            setattr(publisher, k, v)

    # populate relations
    
    if country_codes:
        try:
            print('g')
            links = sqlapp.session.query(CountryCode).filter(CountryCode.code.in_(country_codes)).all()
            print(links)
            setattr(publisher, 'country_codes', links)
        except:
            print('Failed to build Publisher <=> CountryCode relationship')
            raise
    # O2M
    if pm_fields:
        pm_fields_instances = [ PublisherField(field_name=field, publisher_name=id)  for field in pm_fields ]
        setattr(publisher, 'pm_fields', pm_fields_instances)
    
    sqlapp.session.commit()


## RELATIONSHIP PATTERN

Always use `back_populates` arg instead of `back_ref` . if the relation is bi directional

TODO: construct an example

In [22]:
updated_p = {
    'country_codes':['FR', 'IT']
}

updatePublisher('partoo', **updated_p)

p = sqlapp.session.query(Publisher).filter_by(name='partoo').first()
p.country_codes

g
[<CountryCode(id=3, code=AvailableCountryCodes.FR)>, <CountryCode(id=4, code=AvailableCountryCodes.IT)>]


[<CountryCode(id=3, code=AvailableCountryCodes.FR)>,
 <CountryCode(id=4, code=AvailableCountryCodes.IT)>]

In [12]:
try:
    CountryCodeRepository().filter_by(code__in=['ES', 'IT', 'KJ']).all()
except Exception as e:
    print(e.message)

(exceptions.ValueError) KJ is not a valid AvailableCountryCodes


/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
  after removing the cwd from sys.path.


In [ ]:
import logging
log = logging.getLogger(__name__)

def update_country_codes(instance, values):
    try:
        links = CountryCodeRepository().filter_by(code__in=values).all()
        setattr(instance, 'country_codes', links)
    except Exception as e:
        log('Failed to build Publisher <=> CountryCode relationship')

In [13]:
import enum
class AvailableCountryCodes(enum.Enum):
    FR = 'FR'
    EN = 'EN'
    IT = 'IT'
    ES = 'ES'

In [14]:
for x in AvailableCountryCodes:
    print(x)

AvailableCountryCodes.EN
AvailableCountryCodes.ES
AvailableCountryCodes.FR
AvailableCountryCodes.IT
